# Example Sheet. Chapter 3: Getting Data into DataFrames

**Book: From Social Science to Data Science** 

**Author: Bernie Hogan**

**Last revision: September 19, 2019**

# Importing Data to a DataFrame

In [ ]:
import os 
filein = open("..{0}Data{0}test.txt".format(os.sep))
print(filein.read())

In [ ]:
# Testing for your own operating system:
# Which will work? 

try: 
    filein = open("..\Data\test.txt")
    print(filein.read())
except FileNotFoundError:
    print("If you are on windows then your files are not in the right folder")
    print("If you are on Mac or Linux, then disregard.")
    
try: 
    filein = open("../Data/test.txt")
    print(filein.read())
except FileNotFoundError:
    print("If you are on Mac or Linux then your files are not in the right folder")
    print("If you are on Windows, then disregard.")

# JSON - JavaScript Object Notation

In [ ]:
import json 
import os 

filein = json.loads(open("..{}Data{}muppetEpisodes.json".format(os.sep,os.sep)).read())

print(type(filein)) # This shows it is a dictionary, so let's ask for keys. 
      
print(filein.keys()) # Perhaps we want to explore the 'data' key. 

print(type(filein['data'])) # It would appear 'data' is a list. 

print(len(filein['data'])) # This list has 100 entries. 

print(filein['data'][0].keys()) # Inspect the keys - these will go in our table.

In [ ]:
print(filein['data'][0]) # Let's view the first entry. It's very long with a summary and other details.

Below we will normalise the JSON so that it can be used as a table. We will display the table here on the screen but you will notice that it is too long for the screen size. Below that we will look at the column headers and then select a smaller number of them to display. 

In [ ]:
from pandas.io.json import json_normalize

muppetjson = json.loads(open("..{0}Data{0}muppetEpisodes.json".format(os.sep)).read())
muppetdf = json_normalize(muppetjson["data"])
display(muppetdf)

In [ ]:
for c,i in enumerate(muppetdf.columns):
    print(c,i)

In [ ]:
display(muppetdf.iloc[:,[7,13,27]].head())

# Markup languages: HTML and XML

In [ ]:
import bs4,os

wikiHTML = open("..{0}Data{0}Canada_Wiki.html".format(os.sep),'r').read()
print(len(wikiHTML))

In [ ]:
print(wikiHTML[:200])

# Using BeautifulSoup

In [ ]:
# Step 1. Make the soup 
soup = bs4.BeautifulSoup(wikiHTML, 'html.parser')

# Query the soup
print(soup.title.text)
links = soup.find_all("a")
print(len(links))

In [ ]:
urls = []
internal_links = []

for souplink in soup.find_all('a'):
    link = souplink.get('href')
    if link: # That means the link is a hypertext reference and not a section heading
        if 'http' in link:
            urls.append(link)
        else:
            internal_links.append(link) 
    else:
        print(souplink)

print(len(urls),len(internal_links))

In [ ]:
for i in internal_links[:10]: print(i)

In [ ]:
import pandas as pd 

wikiLinks = pd.DataFrame(internal_links,columns=["internal_links"])

def get_wiki(text):
    if text[:5] == "/wiki": return True
    else: return False
    
wikiLinks["wiki"] = wikiLinks["internal_links"].map(lambda x: get_wiki(x))
wikiLinks.head(10)

print("There are {} internal links on this page, {} of which are unique, and {} of which are to other wiki pages".format( 
        len(wikiLinks["internal_links"]), 
        len(wikiLinks["internal_links"].unique() ),
        len(wikiLinks[wikiLinks["wiki"]]) #Notice here I sliced to only "wiki" == True.
        ))

In [ ]:
soup = bs4.BeautifulSoup(wikiHTML,'lxml') #res.content
tables = soup.find_all('table')[0] 
parsed_tables = pd.read_html(str(tables)) # This will return a list of DataFrames, one for each table detected.
print(len(parsed_tables)) # This will show us there is only one table detected. 
display(parsed_tables[0].head())

# XML 

In [ ]:
# loading some xml
import bs4, os

infile = open("..{0}Data{0}Canada.xml".format(os.sep),'r')

wikitext = infile.read()

# Note: In some circumstances, the file is saved as encoded data, in which case
# use the .decode('utf-8') function on the text. As in:
# soup = bs4.BeautifulSoup(wikitext.decode('utf8'), "lxml")
soup = bs4.BeautifulSoup(wikitext, "lxml")

print (soup.mediawiki.page.revision.id )

In [ ]:
sep = "\n~~~~~\n"

for i in soup.children: print(i.name)
print(sep)
for i in soup.html.children: print(i.name)
print(sep)
for i in soup.html.body.children: print(i.name)
print(sep)
for i in soup.mediawiki.children: print(i.name) 
print(sep)
for i in soup.mediawiki.page.children: print(i.name)
print(sep)
# I discover that we can just say soup.page and it will get the text. 
y = soup.page.text

print (soup.page.text == soup.html.body.mediawiki.page.text)
print(y[:100],"...")

# CSV

In [ ]:
import csv,os

with open('..{0}Data{0}MuppetsTable.csv'.format(os.sep), newline='') as file_to_read:
    filereader = csv.reader(file_to_read, delimiter=',', quotechar='|')
    for row in filereader:
        row = ["{:<20}".format(x) for x in row]
        print("".join(row))

In [ ]:
import pandas as pd

df = pd.read_csv('..{0}Data{0}MuppetsTable.csv'.format(os.sep))
df

In [ ]:
help(pd.read_csv)

# Excel

In [ ]:
import pandas as pd, os 

mt = pd.read_excel("..{0}Data{0}MuppetsTable.xlsx".format(os.sep))
display(mt)

# Pickling

In [ ]:
import pickle

x = ['1','2']
pickle.dump(x,open("temp.txt",'wb'))
y = pickle.load(open("temp.txt",'rb'))
print(y)